<a href="https://colab.research.google.com/github/isasenth/cdle_youth_handson_nlp_1/blob/okazawa/visualize_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 可視化用パッケージ
!pip install nlplot

# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

# 辞書のパスの確認
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


# データ読み込み

In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/nishika_fake-news/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/nishika_fake-news/test.csv')

train_df

,id,isFake,text
0,d19828eb64,1,Cによると、アメリカの元大統領で、最長寿だったジョージ・ウォーカー・ブッシュ氏が27日(C-...
1,dfaab096bd,0,中日新聞によると、コナミカップ・プロ野球アジアシリーズ2007の決勝戦・日本の中日ドラゴンズ...
2,163504bf95,1,愛媛Cは、11月12日にリーグ準加盟の承認を受けて、来期リーグ加盟を目指す愛媛Cに対して、鈴...
3,ed3c9dc579,0,国民日報によると3日、7時50分（UTC+9、日本時間と同じ）大韓民国京畿道平沢市の西海岸（...
4,e06f88267f,1,共同通信によると、5日午後2時過ぎから東京都、神奈川県、千葉県の3都県の広い範囲の地域で停電...
...,...,...,...
3776,d65c42f576,1,読売新聞によると、1月19日未明3時10分ごろ大分市の建設コンサルタント会社社長宅が全焼する...
3777,a478642504,0,シリアのガジ・カナン内相が12日自殺したとシリアの国営通信社SANAが同日伝えた。自殺は大き...
3778,a428af48d0,1,朝日新聞によると、日本陸上連盟は12月18日に行った理事会で、女子単独の国際大会となる北京オ...
3779,347213dd2b,1,Cによると、中国のインターネット中毒が中国で正式に病気として認定されるのは来年になる見通しで...


train, testともに全3781データ

In [ ]:
train_df['isFake'].value_counts()

1    1937
0    1844
Name: isFake, dtype: int64

訓練データはフェイクニュースのほうが僅かに多い

# 形態素解析

In [ ]:
import MeCab

mecab = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
wakati = MeCab.Tagger('-Owakati d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

def extract_nouns(text):
    nouns = []
    node = mecab.parseToNode(text)
    while node:
        features = node.feature.split(',')
        if features[0] in ['名詞']:
            nouns.append(node.surface)
        node = node.next

    return ' '.join(nouns)

In [ ]:
train_df['nouns'] = train_df['text'].apply(lambda s: extract_nouns(s))
train_df['wakati'] = train_df['text'].apply(lambda s: wakati.parse(s).strip('\n'))
train_df.head()

,id,isFake,text,nouns,wakati
0,d19828eb64,1,Cによると、アメリカの元大統領で、最長寿だったジョージ・ウォーカー・ブッシュ氏が27日(C-...,C アメリカ 元大統領 長寿 ジョージ ウォーカー ブッシュ 氏 27日 C-5 ロサンゼル...,C に よる と 、 アメリカ の 元 大統領 で 、 最 長寿 だっ た ジョージ ・ ウ...
1,dfaab096bd,0,中日新聞によると、コナミカップ・プロ野球アジアシリーズ2007の決勝戦・日本の中日ドラゴンズ...,中日新聞 コナミカップ プロ野球 アジアシリーズ2007 決勝戦 日本 中日ドラゴンズ 対 ...,中日 新聞 に よる と 、 コナミ カップ ・ プロ 野球 アジア シリーズ 2007 の...
2,163504bf95,1,愛媛Cは、11月12日にリーグ準加盟の承認を受けて、来期リーグ加盟を目指す愛媛Cに対して、鈴...,愛媛 C 11月12日 リーグ 加盟 承認 来期 リーグ 加盟 愛媛 C 鈴木 チェアマン ...,愛媛 C は 、 11 月 12 日 に リーグ 準 加盟 の 承認 を 受け て 、 来期...
3,ed3c9dc579,0,国民日報によると3日、7時50分（UTC+9、日本時間と同じ）大韓民国京畿道平沢市の西海岸（...,国民日報 3日 7時 50分 UTC 9 日本時間 大韓民国 京畿道 平沢市 西海岸 ソヘア...,国民 日報 に よる と 3 日 、 7 時 50 分 （ UTC + 9 、 日本 時間 ...
4,e06f88267f,1,共同通信によると、5日午後2時過ぎから東京都、神奈川県、千葉県の3都県の広い範囲の地域で停電...,共同通信 5日 午後2時 過ぎ 東京都 神奈川県 千葉県 3 都県 範囲 地域 停電 東京電...,共同 通信 に よる と 、 5 日 午後 2 時 過ぎ から 東京 都 、 神奈川 県 、...


In [ ]:
# 名詞の数
train_df['num_nouns'] = train_df['nouns'].apply(lambda s: len(s.split()))
# トークンの数
train_df['num_tokens'] = train_df['wakati'].apply(lambda s: len(s.split()))
train_df[['num_nouns', 'num_tokens']].describe()

,num_nouns,num_tokens
count,3781.000000,3781.000000
mean,104.600635,323.007934
std,53.389472,162.693909
min,4.000000,22.000000
25%,68.000000,210.000000
50%,96.000000,300.000000
75%,127.000000,392.000000
max,588.000000,1701.000000


トークンの数の最大値が1701であるため，bertにそのまま入力すると情報が落ちる

# 可視化

## トークン数

In [ ]:
import plotly.express as px

fig = px.histogram(train_df, x="num_tokens", color='isFake')
fig.show()

In [ ]:
import nlplot

# 全データ
npt_noun = nlplot.NLPlot(train_df, target_col='nouns')
npt_all = nlplot.NLPlot(train_df, target_col='wakati')

# wikinews
npt_wiki_noun = nlplot.NLPlot(train_df.query('isFake == 0'), target_col='nouns')
npt_wiki_all = nlplot.NLPlot(train_df.query('isFake == 0'), target_col='wakati')

# フェイクニュース
npt_fake_noun = nlplot.NLPlot(train_df.query('isFake == 1'), target_col='nouns')
npt_fake_all = nlplot.NLPlot(train_df.query('isFake == 1'), target_col='wakati')

## 名詞に着目


In [ ]:
fig_unigram_tree_wiki = npt_wiki_noun.treemap(
    title='wikinews',
    ngram=1,
    top_n=50,
)
fig_unigram_tree_fake = npt_fake_noun.treemap(
    title='フェイクニュース',
    ngram=1,
    top_n=50,
)

fig_unigram_tree_wiki.show()
fig_unigram_tree_fake.show()

100%|██████████| 1937/1937 [00:00<00:00, 7802.06it/s]


- フェイクニュースは新聞社の名前（朝日新聞，読売新聞など）
- 日本の出現回数がwikinewsと比較すると2倍以上

## フェイクニュース特有の表現
単語単位でなくひとかたまりで確認

不自然な表現ある？

In [ ]:
# stopwordsを設定（設定しないと，文の終わりが多くでてきてしまうため）
top_n = 20
stopwords = npt_all.get_stopword(top_n=top_n, min_freq=0)

npt_all.bar_ngram(
    title='stopwords',
    xaxis_label='word_count',
    yaxis_label='word',
    ngram=1,
    top_n=top_n,
)

100%|██████████| 3781/3781 [00:01<00:00, 2623.38it/s]


In [ ]:
from plotly.subplots import make_subplots

ngram = 5
top_n = 50

# wikinewsのfigを取得
fig_unigram_wiki = npt_wiki_all.bar_ngram(
    xaxis_label='word_count',
    yaxis_label='word',
    ngram=ngram,
    top_n=top_n,
    stopwords=stopwords,
)

# フェイクニュースのfigを取得
fig_unigram_fake = npt_fake_all.bar_ngram(
    xaxis_label='word_count',
    yaxis_label='word',
    ngram=ngram,
    top_n=top_n,
    stopwords=stopwords,
)

# subplot
trace1 = fig_unigram_wiki['data'][0]
trace2 = fig_unigram_fake['data'][0]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('wikinews', 'フェイクニュース'),
    shared_xaxes=True)

fig.update_xaxes(title_text='word count', row=1, col=1)
fig.update_xaxes(title_text='word count', row=1, col=2)

fig.update_layout(
    height=1100, width=1900,
    title_text='unigram wikinews vs. フェイクニュース')

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)

# plotly.offline.plot(
#     fig, filename='unigram wikinews vs. フェイクニュース', auto_open=False)

fig.show()

100%|██████████| 1937/1937 [00:00<00:00, 3130.62it/s]


- 数字は前処理したほうが良いかも
- フェイクニュースでないものは時間の情報が多い
- フェイクニュースは無意味な繰り返しが多い
- フェイクニュースには「ーーーーー」のようなものが見られる